# Task 3 - Model Explainability (SHAP)

**Objective:** Interpret the decisions made by the fraud detection models using SHAP (SHapley Additive exPlanations).

This notebook will cover:
- Feature importance globally
- Local explanations for specific fraud vs. legitimate transactions
- Visualizing the drivers of fraud detection